In [39]:
import plotly.express as pe
import xarray as xr
import dask.array as da
from dask.distributed import Client, LocalCluster
import scipy.stats as stats
import numpy as np

In [40]:
client = Client("tcp://127.0.0.1:42989")

In [105]:
file_params = '/home/davidbyrne/disks/ssd200/wetbulb/analysis/gevfit_bm_1deg.nc'
file_extremes = '/home/davidbyrne/disks/ssd200/test.nc'

In [106]:
ds_params = xr.open_dataset(file_params).transpose('param','lat','lon')
params = da.from_array( ds_params['test_data'].data )

In [107]:
def return_levels(params, distribution, return_periods):
    return_periods = np.array(return_periods)
    params = np.array(params)
    return_levels = distribution.isf(1/return_periods, params[0], params[1], params[2])
    return return_levels

def return_levels_chunk(chunk, distribution, return_periods):
    n_p, n_r, n_c = chunk.shape
    chunkF = chunk.reshape((n_p, n_r*n_c))
    n_pts = n_r*n_c
    n_rp = len(return_periods)
    out_array = np.zeros((n_rp, n_pts))*np.nan
    
    for pt in range(n_pts):
        params = chunkF[:, pt]
        if len(params)>2:
            rl = return_levels(params[:3], distribution, return_periods)
            out_array[ :, pt] = rl
        
    return out_array.reshape((n_rp, n_r, n_c))

def return_periods(params, distribution, return_levels, rpmax=100, omit_above=200):
    return_levels = np.array(return_levels)
    params = np.array(params)
    return_periods = 1-distribution.cdf(return_levels, params[0], params[1], params[2])
    #return_periods[return_periods==0] = np.nan
    #return_periods = 1/return_periods
    #return_periods[return_periods>omit_above] = np.nan
    #return_periods[return_periods>rpmax] = rpmax
    return return_periods

def return_periods_chunk(chunk, distribution, return_levels):
    n_p, n_r, n_c= chunk.shape
    print(chunk.shape)
    chunkF = chunk.reshape((n_p, n_r*n_c))
    n_pts = n_r*n_c
    n_rp = len(return_levels)
    out_array = np.zeros((n_rp, n_pts))*np.nan
    
    for pt in range(n_pts):
        params = chunkF[:, pt]
        if len(params)>2:
            rp = return_periods(params[:3], distribution, return_levels)
            out_array[:, pt] = rp
        
    return out_array.reshape((n_rp, n_r, n_c))

In [108]:
params = params.rechunk([-1, 50, 50])

In [103]:
%%time
rl = da.map_blocks(return_levels_chunk, params, stats.genextreme, [1,2,5,10,50,100,200])
rl = rl.compute()
rp = da.map_blocks(return_periods_chunk, params, stats.genextreme, [20, 25, 30, 32,35])
rp = rp.compute()

CPU times: user 31.9 ms, sys: 973 µs, total: 32.9 ms
Wall time: 5.01 s


In [100]:
ds_out = xr.Dataset( coords )

(0, 0, 0)
(1, 1, 1)
CPU times: user 19.2 ms, sys: 11.9 ms, total: 31.1 ms
Wall time: 4.87 s
